<img style="float: center;" src='https://github.com/spacetelescope/jwst-pipeline-notebooks/raw/main/_static/stsci_header.png' alt="stsci_logo" width="900px"/>

In [ ]:
%matplotlib notebook

# RW-DDT Flux-Calibrated Analysis Template Notebook

**Authors**: Taylor James Bell (ESA/AURA for STScI)<br>
**Last Updated**: June 05, 2025<br>
**jwst Pipeline Version**: 1.18.0 (Build 11.3)<br>
**Eureka! Pipeline Version**: https://github.com/kevin218/Eureka/tree/tjb_rwddt

Note that additional contextual information can be found in `README_Fluxcal.md`

**Purpose**:<br/>

It should not be necessary to edit any cells other than in the [1. Define your eventlabel and top directory](#1.-Define-your-eventlabel-and-top-directory) section and the [4.1 Setting up the Stage 4cal ECF](#4.1-Setting-up-the-Stage-4cal-ECF) subsection unless you want to manually explore/optimize different data processing steps.

The methods of this notebook were based on the findings of [Gordon+2025](https://iopscience.iop.org/article/10.3847/1538-3881/ad8cd4)

**Data**:<br/>
This notebook assumes the Stage 1 rateints files have already been downloaded from MAST using the `rocky-worlds-utils/download_JWST.py` script.

**JWST pipeline version and CRDS context**:<br/>
This notebook was written for the calibration pipeline version given above and uses the context associated with this version of the JWST Calibration Pipeline. Information about this and other contexts can be found in the JWST Calibration Reference Data System (CRDS) [server]((https://jwst-crds.stsci.edu/)). If you use different pipeline
versions, please refer to the table [here](https://jwst-crds.stsci.edu/display_build_contexts/) to determine what context to use. To learn more about the differences for the pipeline, read the relevant [documentation](https://jwst-docs.stsci.edu/jwst-science-calibration-pipeline/jwst-operations-pipeline-build-information)

## Table of Contents
- [0. Importing the required components](#0.-Importing-the-required-components)
- [1. Define your eventlabel and top directory](#1.-Define-your-eventlabel-and-top-directory)
- [2. Stage 2](#2.-Stage-2)
- [3. Stage 3 - Pixels to Lightcurve](#3.-Stage-3---Pixels-to-Lightcurve)
- [4. Stage 4cal - Flux Calibration](#4.-Stage-4cal---Flux-Calibration)
- [5. Logging in-eclipse absolutely-calibrated stellar flux level](#5.-Logging-in-eclipse-absolutely-calibrated-stellar-flux-level)
- [6. Sanity-checking approximate eclipse depth from absolutely-calibrated data](#6.-Sanity-checking-approximate-eclipse-depth-from-absolutely-calibrated-data)

## 0. Importing the required components

There should be no need to change any of this

In [ ]:
# Importing a bunch of Eureka! components
import eureka.lib.plots
import eureka.S1_detector_processing.s1_process as s1
import eureka.S2_calibrations.s2_calibrate as s2
import eureka.S3_data_reduction.s3_reduce as s3
import eureka.S4cal_StellarSpectra.s4cal_StellarSpec as s4cal

# Set up some parameters to make plots look nicer. You can set usetex=True if you have LaTeX installed
eureka.lib.plots.set_rc(style='eureka', usetex=False, filetype='.png')

# Some imports to interact with outputs within the Jupyter notebook
from IPython.display import Image, display
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
plt.ioff()

## 1. Define your eventlabel and top directory

Next, we need to choose a short, meaningful label (without spaces) that describes the data we're currently working on. This eventlabel will determine will give nicknames to all your output folders and files.

We also need to tell the notebook where all our data is going to be stored

In [ ]:
# Enter in a custom eventlabel that will be used to distinguish the outputs
# from all subsequent processing
eventlabel = '' ## <- ENTER YOUR LABEL HERE

# Specify here the top directory that will contain all ingested and output files
topdir = '' ## <- ENTER YOUR TOPDIR HERE

## 2. Stage 2

### 2.1 Setting up the Stage 2 ECF

For flux-calibrated analyses, there is no need to change any of the Stage 2 settings beyond what is provided in the template, so we'll just read those defaults in.

In [ ]:
s2_ecf_contents = f"""# Eureka! Control File for Stage 2: Data Reduction

# Stage 2 Documentation: https://eurekadocs.readthedocs.io/en/latest/ecf.html#stage-2

pmap				1364

skip_flat_field     False
skip_photom         False

# Project directory
topdir              {topdir}

# Directories relative to topdir
inputdir            Analysis_A/Quicklook/MAST_Stage1
outputdir           Analysis_A/Fluxcal/Stage2
"""

# This will save the ECF as a file that the next cell can read-in
with open(f'./S2_{eventlabel}.ecf', 'w') as f:
    f.write(s2_ecf_contents)

### 2.2 Running Stage 2

Here we run the Eureka! Stage 2 pipeline using the settings we defined above. This should take <1 minute, but that will depend on the data volume of the observation you're working on and the specifics of your CPU

If you previously ran Stage 2 and want to re-use those outputs, you can just comment-out the following line

In [ ]:
s2_meta = s2.calibrateJWST(eventlabel)

## 3. Stage 3 - Pixels to Lightcurve

### 3.1 Setting up the Stage 3 ECF

For flux-calibrated analyses, there is no need to change any of the Stage 3 settings beyond what is provided in the template, so we'll just read those defaults in.

In [ ]:
s3_ecf_contents = f"""# Eureka! Control File for Stage 3: Data Reduction

# Stage 3 Documentation: https://eurekadocs.readthedocs.io/en/latest/ecf.html#stage-3

ncpu               18
max_memory         1.5

pmap               1364

calibrated_spectra True

# Background parameters
ff_outlier         True        # Set False to use only background region (recommended for deep transits)
                               # Set True to use full frame (works well for shallow transits/eclipses)
bg_thresh          [5,5]
interp_method      linear      # Interpolate bad pixels. Options: None (if no interpolation should be performed), linear, nearest, cubic

# Centroiding parameters
centroid_method    mgmc        # Method used for centroiding. Options: mgmc, fgc
ctr_guess		   fits    	   # Initial guess of centroid position. If None, will first perform centroiding on whole frame (can sometimes fail)

# Photometric extraction parameters
phot_method        photutils   # photutils (aperture photometry using photutils), poet (aperture photometry using code from POET), or optimal (for optimal photometric extraction)
aperture_edge      exact       # center (pixel is included only if its center lies within the aperture), or exact (pixel is weighted by the fractional area that lies within the aperture)
photap             5.69        # Size of photometry aperture radius in pixels
skyin              8.63        # Inner sky annulus edge, in pixels
skywidth           2.82        # Width of the sky annulus, in pixels

# Diagnostics
nplots             3

# Project directory
topdir             {topdir}

# Directories relative to topdir
inputdir           Analysis_A/Fluxcal/Stage2
outputdir          Analysis_A/Fluxcal/Stage3
"""

# This will save the ECF as a file that the next cell can read-in
with open(f'./S3_{eventlabel}.ecf', 'w') as f:
    f.write(s3_ecf_contents)

### 3.2 Running Stage 3

Here we run the Eureka! Stage 3 pipeline using the settings we defined above. This should take <1 minute, but that will depend on the data volume of the observation you're working on and the specifics of your CPU

If you previously ran Stage 3 and want to re-use those outputs, you can just comment-out the following line

In [ ]:
spec, s3_meta = s3.reduce(eventlabel)

### 3.3 Sanity checking the Stage 3 output figures

Let's look at the plot outputs from Stage 3 to make sure they look reasonable (similar to the checks from the quicklook notebook)

In [ ]:
figures = np.sort(glob(f'{s3_meta.outputdir}/figs/*'))
for figure in figures:
    print(figure)
    display(Image(filename=figure, embed=True, width=700))

## 4. Stage 4cal - Flux Calibration

In Stage 4cal, we will work on removing time-series outliers, applying a finite aperture correction, and getting absolutely-calibrated stellar flux levels from the in-eclipse data

### 4.1 Setting up the Stage 4cal ECF

You will need to set several settings here that are specific to your planetary system. Specifically, you will need to set:

* `t0`; **mid-eclipse time** in units of BMJD_TDB. Copy this value from the results of your best-fit from your deep-dive analysis.
* `rprs`; planet-star radius ratio. Copy this value from your EPF file.
* `period`; orbital period (in days). Copy this value from your EPF file.
* `inc`; orbital inclination (in degrees). Copy this value from your EPF file.
* `ars`; semi-major axis to stellar radius ratio. Copy this value from your EPF file.
* `base_dur`; out-of-eclipse baseline duration (in days) to use when computing the baseline flux. This value isn't super important for eclipse observations, but we will use it later as a sanity check. Make it as large as reasonable to get a lower error, but keep it small enough to avoid the baseline curvature at the start of the observations. It's okay if this takes some trial and error. A value of 0.03 may suffice

In [ ]:
s4_ecf_contents = f"""# Eureka! Control File for Stage 4cal: Calibrated Stellar Spectra

# Stage 4cal Documentation: https://eurekadocs.readthedocs.io/en/latest/ecf.html#stage-4cal

# Transit/Eclipse time
t0              ## <- ENTER YOUR VALUE HERE; Time of mid-eclipse (in BMJD_TDB)

# Orbital parameters
rprs            ## <- ENTER YOUR VALUE HERE; Planet-star radius ratio
period          ## <- ENTER YOUR VALUE HERE; Orbital period (in Days)
inc             ## <- ENTER YOUR VALUE HERE; Orbital inclination (in degrees)
ars             ## <- ENTER YOUR VALUE HERE; Semi-major axis to stellar radius ratio

# Light curve to be used before t1 and after t4 for the baseline flux,
# which includes flux from (t1 - base_dur) to t1 and t4 to (t4 + base_dur).
base_dur        0.03  ## <- UPDATE IF NEEDED; Desired out-of-eclipse baseline duration (units of days)

# Correction for extrapolating finite aperture to infinite aperture
apcorr          1.497  ## <- DO NOT TOUCH (Value based on Gordon+2025)

# Outlier detection
sigma_thresh    [4,4,4]  # Three rounds of 4-sigma clipping ([4,4,4])

# Diagnostics
nbin_plot       100  # The number of time bins that should be used for figure 4202
hide_plots      True  # If True, plots will automatically be closed rather than popping up

# Project directory
topdir          {topdir}

# Directories relative to topdir
inputdir        Analysis_A/Fluxcal/Stage3
outputdir       Analysis_A/Fluxcal/Stage4
"""

# This will save the ECF as a file that the next cell can read-in
with open(f'./S4cal_{eventlabel}.ecf', 'w') as f:
    f.write(s4_ecf_contents)

### 4.2 Running Stage 4cal

Here we run the Eureka! Stage 4cal pipeline using the settings we defined above. This should take << 1 minute.

In [ ]:
s4cal_meta, spec, ds = s4cal.medianCalSpec(eventlabel)

### 4.2 Sanity checking the Stage 4cal output figures

Let's look at the regions used for in-eclipse and baseline flux measurements to make sure they look reasonable

In [ ]:
figures = np.sort(glob(f'{s4cal_meta.outputdir}/figs/fig4202*'))
for figure in figures:
    display(Image(filename=figure, embed=True, width=700))

## 5. Logging in-eclipse absolutely-calibrated stellar flux level

The below cell will also include the required error inflation from [Gordon+2025](https://iopscience.iop.org/article/10.3847/1538-3881/ad8cd4)

In [ ]:
stellar_flux = ds.ecl_flux.data[0][0]
# Inflate stellar flux uncertainty to account for calibration systematic uncertainty
stellar_flux_err = np.sqrt(ds.ecl_ferr.data[0][0]**2 + (0.0048*ds.ecl_flux.data[0][0])**2 + (0.0045*ds.ecl_flux.data[0][0])**2)
print(f'Measured Stellar Flux: {stellar_flux:.5f} ± {stellar_flux_err:.5f} mJy')

## 6. Sanity-checking approximate eclipse depth from absolutely-calibrated data

There will be some amount of discrepancy between this value and your best-fit result from your deep-dive analysis, but the estimated eclipse depth from here should at least be vageuly in agreement with the results of your deep-dive analysis

In [ ]:
base_flux = ds.base_flux.data[0][0]
ecl_depth = (base_flux/stellar_flux - 1)*1e6
ecl_err = np.sqrt((ds.base_ferr.data[0][0]/stellar_flux)**2 + (base_flux*ds.ecl_ferr.data[0][0]/stellar_flux**2)**2)*1e6
print(f'Approximate Eclipse Depth: {ecl_depth:.1f} ± {ecl_err:.1f} ppm')